In [2]:
import sys

In [3]:
sys.path

['C:\\Users\\User\\Desktop\\מדמח\\מודלימז\\פרויקט חדש',
 'C:\\Users\\User\\Downloads\\pytorch-normalizing-flows-master\\pytorch-normalizing-flows-master\\nflib',
 'C:\\Users\\User\\try',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\DLLs',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39',
 '',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\plotbrowser-0.1-py3.9.egg',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\win32',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\Pythonwin']

In [4]:
sys.path.append(r"c:\users\user\anaconda3\envs\pytorch_env\lib\site-packages")


In [5]:
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch

import torch.nn as nn
import torch.nn.functional as F
import time
from util import findStatistics
from sklearn.preprocessing import LabelEncoder

In [6]:
df=pd.read_csv(r"C:\Users\User\Desktop\מדמח\מודלימז\פרויקט חדש\churn(3)\telecom_customer_churn.csv")


df.drop('Customer ID',axis = 1, inplace = True)
df['Gender'] = [1 if each == 'Female' else 0 for each in df['Gender']]
df['Married'] = [1 if each == 'Yes' else 0 for each in df['Married']]
df['Churn Category'] = df['Churn Category'].map({'Competitor':1
,'Dissatisfaction':2
,'Attitude':3
,'Price':4
,'Other':5})
df['Churn Reason'] = df['Churn Reason'].map({'Competitor had better devices':1, 'Product dissatisfaction':2,
       'Network reliability':3, 'Limited range of services':4,
       'Competitor made better offer':5, "Don't know":6,
       'Long distance charges':7, 'Attitude of service provider':8,
       'Attitude of support person':9,
       'Competitor offered higher download speeds':10,
       'Competitor offered more data':11,
       'Lack of affordable download/upload speed':12, 'Deceased':13, 'Moved':14,
       'Service dissatisfaction':15, 'Price too high':16,
       'Lack of self-service on Website':17,
       'Poor expertise of online support':18, 'Extra data charges':19,
       'Poor expertise of phone support':20})


df['Customer Status'] = df['Customer Status'].map({'Stayed':1
,'Churned':2
,'Joined':3})


df['Phone Service'] = [1 if each == 'Yes' else 0 for each in df['Phone Service']]
df['Multiple Lines'] = [1 if each == 'Yes' else 0 for each in df['Multiple Lines']]
df['Internet Service'] = [1 if each == 'Yes' else 0 for each in df['Internet Service']]
df['Internet Type'] = [1 if each == 'Yes' else 0 for each in df['Internet Type']]
df['Online Security'] = [1 if each == 'Yes' else 0 for each in df['Online Security']]
df['Online Backup'] = [1 if each == 'Yes' else 0 for each in df['Online Backup']]
df['Device Protection Plan'] = [1 if each == 'Yes' else 0 for each in df['Device Protection Plan']]
df['Premium Tech Support'] = [1 if each == 'Yes' else 0 for each in df['Premium Tech Support']]
df['Streaming TV'] = [1 if each == 'Yes' else 0 for each in df['Streaming TV']]
df['Streaming Movies'] = [1 if each == 'Yes' else 0 for each in df['Streaming Movies']]
df['Streaming Music'] = [1 if each == 'Yes' else 0 for each in df['Streaming Music']]
df['Unlimited Data'] = [1 if each == 'Yes' else 0 for each in df['Unlimited Data']]
df['Paperless Billing'] = [1 if each == 'Yes' else 0 for each in df['Paperless Billing']]
df['Contract'] = df['Contract'].map({'One Year':1, 'Month-to-Month':2, 'Two Year':3})
df['Offer'] = df['Offer'].map({'None':0, 'Offer E':1, 'Offer D':2, 'Offer A':3, 'Offer B':4, 'Offer C':5})
df['Payment Method'] = df['Payment Method'].map({'Bank Withdrawal':1
,'Credit Card':2
,'Mailed Check':3})
# filling the null values with the mean

value = df['Avg Monthly Long Distance Charges'].mean()
values = df['Avg Monthly GB Download'].mean()
val1 = df['Churn Category'].mean()
val2 = df['Churn Reason'].mean()
df['Avg Monthly Long Distance Charges'].fillna(value = value, inplace = True)
df['Avg Monthly GB Download'].fillna(value = values, inplace = True)
df['Churn Category'].fillna(value = val1, inplace = True)
df['Churn Reason'].fillna(value = val2, inplace = True)
df.isnull().sum()

df.drop('City', axis = 1, inplace = True)
x = df.drop('Customer Status', axis = 1)
y = df['Customer Status'].values
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit_transform(x)
X = scaler.fit_transform(x)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)


In [8]:
df

,Gender,Age,Married,Number of Dependents,Zip Code,Latitude,Longitude,Number of Referrals,Tenure in Months,Offer,...,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status,Churn Category,Churn Reason
0,1,37,1,0,93225,34.827662,-118.999073,2,9,0,...,2,65.60,593.30,0.00,0,381.51,974.81,1,2.235955,7.458534
1,0,46,0,0,91206,34.162515,-118.203869,0,9,0,...,2,-4.00,542.40,38.33,10,96.21,610.28,1,2.235955,7.458534
2,0,50,0,0,92627,33.645672,-117.922613,0,4,1,...,1,73.90,280.85,0.00,0,134.60,415.45,2,1.000000,1.000000
3,0,78,1,0,94553,38.014457,-122.115432,1,13,2,...,1,98.00,1237.85,0.00,0,361.66,1599.51,2,2.000000,2.000000
4,1,75,1,0,93010,34.227846,-119.079903,3,3,0,...,2,83.90,267.40,0.00,0,22.14,289.54,2,2.000000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,20,0,0,91941,32.759327,-116.997260,0,13,2,...,2,55.15,742.90,0.00,0,606.84,1349.74,1,2.235955,7.458534
7039,0,40,1,0,95367,37.734971,-120.954271,1,22,2,...,1,85.10,1873.70,0.00,0,356.40,2230.10,2,2.000000,2.000000
7040,0,22,0,0,95432,39.108252,-123.645121,0,2,1,...,2,50.30,92.75,0.00,0,37.24,129.99,3,2.235955,7.458534
7041,0,21,1,0,92075,33.001813,-117.263628,5,67,3,...,2,67.85,4627.65,0.00,0,142.04,4769.69,1,2.235955,7.458534


In [ ]:
X=X.drop(columns=['hazardous'],axis=0)
y=df['hazardous']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

In [22]:
parameters = {'max_depth':list(range(50)), 'min_samples_leaf':list(range(2,50,1)),'criterion':["gini", "entropy"]}

et=time.time()
model= DecisionTreeClassifier()
clf = GridSearchCV(model, parameters)

clf.fit(X_train, y_train)
ft=time.time()

print("elapsed time {}".format(ft-et))
print("score: {}".format(clf.score(X_test,y_test)))

elapsed time 592.2236559391022
score: 1.0


In [23]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(15,32)
        self.fc2=nn.Linear(32,64)
        self.fc6=nn.Linear(64,128)
        self.fc7=nn.Linear(128,256)
        self.fc8=nn.Linear(256,128)
        self.fc3=nn.Linear(128,32)
        self.fc4=nn.Linear(32,8)
        self.fc5=nn.Linear(8,1)


    def forward(self,x):
        
        x = F.relu(self.fc1(x))
        
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = F.relu(self.fc8(x))
        x = F.relu(self.fc3(x))
        
        x = F.relu(self.fc4(x))
        x = F.sigmoid(self.fc5(x))
        return x
    
     
    
    
net = Net()
net.load_state_dict(torch.load(r'C:\Users\User\Desktop\מדמח\מודלימז\פרויקט חדש\batches117'))
#best_max_depth,best_min_samples_leaf,best_criterion=params[np.argmax(scores)]
#best_criterion='entropy' if best_criterion==1 else 'gini'

<All keys matched successfully>

In [24]:
scores=[]

et=time.time()
statistics=np.array(findStatistics(X_train,y_train))
for max_depth in list(range(50)):
    for min_samples_leaf in list(range(2,50,1)):
        for criterion in [1, 2]:
            hyperTensor=np.array([max_depth,min_samples_leaf,criterion])
            combtensor=torch.tensor(np.concatenate((statistics,hyperTensor))).float()
            
            score_predicted=net(combtensor).detach().numpy()
            scores.append(score_predicted)
params=[]
for max_depth in list(range(50)):
    for min_samples_leaf in list(range(2,50,1)):
        for criterion in [1, 2]:
            params.append((max_depth,min_samples_leaf,criterion)) 
            
scores=np.array(scores).flatten()
params=np.array(params)

tenLargetIdx = (-scores).argsort()[:10]
bests=params[tenLargetIdx]   


res=[]
for max_depth,min_samples_leaf,criterion in bests:
    criterion='entropy' if criterion==1 else 'gini'
    model= DecisionTreeClassifier(max_depth=max_depth,min_samples_leaf=min_samples_leaf,criterion=criterion)
    model.fit(X_train,y_train)
    res.append(model.score(X_test,y_test))
    

best=np.argmax(np.array(res))
best_max_depth,best_min_samples_leaf,best_criterion=bests[best]
best_criterion='entropy' if best_criterion==1 else 'gini'
model2= DecisionTreeClassifier(max_depth=best_max_depth,min_samples_leaf=best_min_samples_leaf,criterion=best_criterion)

model2.fit(X_train,y_train)
ft=time.time()
print("elapsed time {}".format(ft-et))

print('score: {}'.format(model2.score(X_test,y_test)))
            


elapsed time 20.51399517059326
score: 1.0


In [17]:
best_max_depth

8